In [66]:
import pandas as pd
import numpy as np
import os.path as op
import os
from scipy.optimize import minimize
from scipy.spatial.distance import cdist
import itertools

from varmodel_utils import objective, get_full_variance_indices,\
                           get_sub_variance_indices, prune

In [3]:
bp = '/data/RocklandSample/derivatives/paper1/'
exp = 'figures'
bpp = op.join(bp, exp)
try:
    os.makedirs(bpp)
except FileExistsError:
    pass

df_pyonly = pd.read_hdf(bp + 'connectomes_mp_pyonly.h5')
df_pyonly['Instrumentation'] = "Data"

df = pd.read_hdf(bp + 'connectomes_mp.h5')
df['Instrumentation'] = "Pipeline"

In [4]:
df = pd.concat([df, df_pyonly], axis=0, ignore_index=True)

In [5]:
cols = ["subject", "session", "directions", "pipeline"]

### Reference graphs
df_ref = prune(df, Instrumentation='Pipeline', simulation="ref")
df_ref = df_ref.reset_index(drop=True).reset_index(level=0).set_index(cols)

### Simulation setting graphs
df_pip = prune(df, Instrumentation='Pipeline')
df_pip = df_pip.reset_index(drop=True).reset_index(level=0).set_index(cols)

df_dat = prune(df, Instrumentation='Data')
df_dat = df_dat.reset_index(drop=True).reset_index(level=0).set_index(cols)

In [72]:
# df_touse = df_ref
# mca_touse = False
# fn = lambda x: np.cov(x, ddof=1)

df_touse = df_dat
mca_touse = True
fn = lambda x: cdist(x, x, metric='euclidean')

gs = np.array([np.reshape(g, -1) for g in df_touse.graph.values])
gs.shape

(4200, 6889)

In [60]:
tmpgs.shape

(168, 6889)

In [74]:
fn(tmpgs).shape

(168, 168)

In [76]:
a = 4 if mca_touse else 3

ests = np.empty((a,1))
for idx, sub in enumerate(df_touse.index.levels[0]):
    tmpdf = df_touse.loc[sub]
    tmpdf['index'] = range(0, len(tmpdf))
    tmpin = get_sub_variance_indices(tmpdf, mca=mca_touse)
    tmpgs = np.array([np.reshape(g, -1) for g in tmpdf.graph.values])

    tmpcov = fn(tmpgs)

    estimate = np.array([np.mean(tmpcov[v]) for k, v in tmpin.items()],)
#     estimate = np.linalg.solve(np.tril(np.ones((a,a))), estimate)
    estimate = np.linalg.solve(np.triu(np.ones((a,a))), estimate)
    estimate = np.reshape(estimate, (a, 1))
    ests = np.append(ests, estimate, axis=1)

splits = list(tmpin.keys())
print(splits)
print(np.mean(ests, axis=1))

/Users/greg/code/env/stability/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['xses', 'xdir', 'xpipe', 'xmca']
[ 3.85356260e+01 -1.31177120e-12  1.25930036e-12  6.43544955e+03]


In [52]:
splits

['session', 'directions', 'pipeline']

In [75]:
ind = get_full_variance_indices(df_touse, mca=mca_touse)

# cov = np.cov(gs, ddof=1)
cov = fn(gs)

[len(v[0]) for k, v in ind.items()]

[17635800, 701400, 348600, 172200, 84000]

In [48]:
estimate = np.array([np.mean(cov[v]) for k, v in ind.items()])

a = 5 if mca_touse else 4
estimate = np.linalg.solve(np.tril(np.ones((a,a))), estimate)

print(estimate)

[ 7.75283250e+03  2.11231794e+01  9.09494702e-13 -9.09494702e-13]


In [35]:
bnds = [(0, None)] * len(estimate)
result = minimize(objective, estimate, (cov, ind), method='L-BFGS-B', options={"disp": True}, bounds=bnds)
print(result)

      fun: 3001.5789222069243
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.01214175, 0.        , 0.        , 0.        ])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 55
      nit: 10
   status: 0
  success: True
        x: array([   0.        , 2423.07039899, 2386.25287929, 2386.25287929])
